In [17]:
# This notebook creates a Multi-Agent solution to write User stories and System tests for a Functional Business Requirement

# CrewAI is used as the Multi-Agent framework 
# Three Agents - Business Analyst, System Tester, System Test Reviewer form the crew
# Three Tasks - Write User Stories, Write System Tests, Review System Tests are performed by each of these agents

# Claude Haiku (Anthropic) is used as the LLM by all these agents

In [18]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [19]:
# Install Crew AI, Tools, Langchain, Google Gemini
!pip install crewai crewai_tools langchain_community langchain_anthropic --quiet
print("---")
!pip show crewai crewai_tools langchain_community langchain_anthropic

---
Name: crewai
Version: 0.28.8
Summary: Cutting-edge framework for orchestrating role-playing, autonomous AI agents. By fostering collaborative intelligence, CrewAI empowers agents to work together seamlessly, tackling complex tasks.
Home-page: 
Author: Joao Moura
Author-email: joao@crewai.com
License: 
Location: /Users/ashwin/anaconda3/lib/python3.11/site-packages
Requires: appdirs, click, embedchain, instructor, langchain, openai, opentelemetry-api, opentelemetry-exporter-otlp-proto-http, opentelemetry-sdk, pydantic, python-dotenv, regex
Required-by: 
---
Name: crewai-tools
Version: 0.1.6
Summary: Set of tools for the crewAI framework
Home-page: 
Author: João Moura
Author-email: joaomdmoura@mgail.com
License: 
Location: /Users/ashwin/anaconda3/lib/python3.11/site-packages
Requires: beautifulsoup4, chromadb, embedchain, lancedb, langchain, openai, pydantic, pyright, pytest, pytube, requests, selenium
Required-by: 
---
Name: langchain-community
Version: 0.0.29
Summary: Community cont

In [20]:
# Necessary imports
import os
from getpass import getpass
from crewai import Agent, Task, Crew
from langchain_anthropic import ChatAnthropic

In [21]:
# Get API Key for Google Gemini
os.environ["ANTHROPIC_API_KEY"] = getpass("Enter ANTHROPIC_API_KEY: ")

llm_claude = ChatAnthropic(model='claude-3-haiku-20240307', temperature=0.8)

Enter ANTHROPIC_API_KEY:  ········


In [22]:
# Create our First Agent
# Role = Business Analyst
# Responsibility = Take a business requirement and come up with a set of Agile user stories 

business_analyst = Agent(
    role="Business Analyst",
    goal="Write an accurate and complete set of agile user stories for the {requirement}",
    backstory="You're working on writing agile user stories "
              "for the requirement: {requirement}. "
              "You capture details in the user story that helps the "
              "tester to ensure the software is built right "
              "and complete in all aspects. "
              "Your work is the basis for "
              "the Tester to write system test cases for this requirement.",
    allow_delegation=False,
	verbose=True,
    llm = llm_claude
)

In [23]:
# Create our Second Agent
# Role = System Tester
# Responsibility = Use the user stories created by the Business Analyst, come up with system test cases to test the user story implementation

system_tester = Agent(
    role="System Tester",
    goal="Write system test cases that will test all functionalities that are implemented through user stories, "
         "which implements the requirement: {requirement}",
    backstory="You're working on creating "
              "system test cases for the requirement: {requirement}. "
              "You base your test cases on the work of "
              "the Business Analyst, who provides the user stories "
              "and relevant information about the requirement. "
              "You follow the main features and "
              "direction of the user story, "
              "as provided by the Content Planner. "
              "You must also ensure to cover boundary conditions and error scenarios "
              "that are relevant to complete test the user stories "
              "provide by the Business Analyst. "
              "You must ensure that the system test cases "
              "provide maxiumum coverage "
              "to test all aspects of the requirement.",
    allow_delegation=False,
    verbose=True,
    llm = llm_claude
)

In [24]:
# Creating our Final Agent
# Role = System Test Reviewer
# Responsibility = Review the system tests created by the System Tester, improve test coverage, scenarios etc.

system_test_reviewer = Agent(
    role="System Test Reviewer",
    goal="Review a given set of System Test Cases to align with "
         "the standards of the organization and coverage to the user stories and requirement: {requirement} ",
    backstory="You are an system test reviewer who receives a set of system test cases "
              "from the System Tester. "
              "Your goal is to review the system test cases "
              "to ensure that it follows industry best practices, organization standards "
              "provides balanced review "
              "when providing improvement or suggestions, "
              "and also ensures highest cover based on the "
              "user stories that are written by the Business Analyst.",
    allow_delegation=False,
    verbose=True,
    llm = llm_claude
)

In [25]:
# Define the task that will be assigned to the Business Analyst

write_stories = Task(
    description=(
        "1. Understand the functional business requirement "
            "and relevant context on the requirement {requirement}.\n"
        "2. Identify the users or business stakeholders involved in the requirement considering "
            "their interests and outcomes relevant to the requirement.\n"
        "3. Develop a set of agile user stories that wil help to "
            "implement the requirement as a software product.\n"
        "4. Include assumptions, acceptance criteria, error conditions as part of the generated user stories."
    ),
    expected_output="A comprehensive set of agile user stories using As [a user persona], I want [to perform this action] so that [I can accomplish this goal] format "
        "with description, outcomes, acceptance criteria, "
        "assumptions, and error conditions.",
    agent=business_analyst,
)

In [26]:
# Define the task that will be assigned to the Content Writer

write_tests = Task(
    description=(
        "1. Use the user stories created by Business Analyst to create a set of "
            "System test cases for the requirement {requirement}.\n"
        "2. Ensure coverage for all scenarios described in the user stories.\n"
		"3. Error conditions and edge cases must be captured "
            "to ensure completeness with the tests.\n"
        "4. Ensure the test cases are structured as "
            "test case id, test description, pre-conditions, test steps, expected results, "
            "test data, actual results and test status.  Leave actual results and test status empty, to be filled during execution\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written set of system test cases "
        "that covers all functionalities described in the user stories, "
        "with a well-defined test description, pre-conditions, test steps, expected results, test data, actual results and test status.",
    agent=system_tester,
)

In [27]:
# Define the task that will be assigned to the Content Editor

review_tests = Task(
    description=("Review the system test cases generated by System Tester for "
                 "completeness, coverage against user stories and "
                 "conformance to the requirement {requirement}"),
    expected_output="A well-written and reviewed system test cases in markdown format, "
                    "ready for test execution. ",
    agent=system_test_reviewer
)

In [28]:
# Bring all agents and tasks together into a Crew

crew = Crew(
    agents=[business_analyst, system_tester, system_test_reviewer],
    tasks=[write_stories, write_tests, review_tests],
    verbose=2
)

2024-11-25 16:44:21,555 - 8401596992 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed
2024-11-25 16:44:23,856 - 6263435264 - __init__.py-__init__:362 - ERROR: Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Users/ashwin/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/ashwin/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/ashwin/anaconda3/lib/python3.11/site-packages/urllib3/connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ashwin/anaconda3/lib/python3.11/http/client.py", line 1378, in getresponse
    response.begin()
  File "/Users/ashwin/anaconda3/lib/python3.11/http/

In [29]:
# Run the tasks in sequence for the topic provided here

result = crew.kickoff(inputs={"requirement": "The system must allow blog visitors to sign up for the newsletter by leaving their email."})

 [DEBUG]: == Working Agent: Business Analyst
 [INFO]: == Starting Task: 1. Understand the functional business requirement and relevant context on the requirement The system must allow blog visitors to sign up for the newsletter by leaving their email..
2. Identify the users or business stakeholders involved in the requirement considering their interests and outcomes relevant to the requirement.
3. Develop a set of agile user stories that wil help to implement the requirement as a software product.
4. Include assumptions, acceptance criteria, error conditions as part of the generated user stories.


> Entering new CrewAgentExecutor chain...
Thought: I now have a clear understanding of the functional business requirement and the relevant context. I have identified the users or business stakeholders involved in the requirement, considering their interests and outcomes relevant to the requirement. I am ready to develop a set of agile user stories that will help to implement the requirement

In [30]:
from IPython.display import Markdown
Markdown(result)

# System Test Cases

## Test Case ID: TC_001
**Test Description:** Verify that the newsletter sign-up form is clearly visible and accessible on the blog.
**Pre-Conditions:**
- The blog homepage is loaded.
- The user has not signed up for the newsletter.
**Test Steps:**
1. Navigate to the blog homepage.
2. Verify that the newsletter sign-up form is prominently displayed on the page.
3. Verify that the sign-up form is easily accessible (e.g., not hidden or requires multiple clicks to find).
**Expected Results:**
- The newsletter sign-up form is clearly visible and easily accessible on the blog homepage.
**Test Data:** N/A
**Actual Results:**
**Test Status:**

## Test Case ID: TC_002
**Test Description:** Verify that the newsletter sign-up form only requires the visitor's email address.
**Pre-Conditions:**
- The blog homepage is loaded.
- The newsletter sign-up form is visible.
**Test Steps:**
1. Navigate to the newsletter sign-up form.
2. Verify that the form only requires the visitor to enter their email address.
3. Verify that there are no additional required fields (e.g., name, phone number).
**Expected Results:**
- The newsletter sign-up form only requires the visitor's email address.
**Test Data:** N/A
**Actual Results:**
**Test Status:**

## Test Case ID: TC_003
**Test Description:** Verify that the system validates the email address format and provides feedback on invalid entries.
**Pre-Conditions:**
- The blog homepage is loaded.
- The newsletter sign-up form is visible.
**Test Steps:**
1. Navigate to the newsletter sign-up form.
2. Enter an invalid email address (e.g., "invalid_email") in the email field.
3. Submit the form.
4. Verify that the system detects the invalid email format and provides feedback to the user.
5. Enter a valid email address (e.g., "valid_email@example.com") in the email field.
6. Submit the form.
7. Verify that the system accepts the valid email address without any error messages.
**Expected Results:**
- The system validates the email address format and provides clear feedback to the user when an invalid email is entered.
- The system accepts the email address when a valid format is entered.
**Test Data:**
- Invalid email address: "invalid_email"
- Valid email address: "valid_email@example.com"
**Actual Results:**
**Test Status:**

## Test Case ID: TC_004
**Test Description:** Verify that the system confirms the successful sign-up and provides instructions on how to unsubscribe.
**Pre-Conditions:**
- The blog homepage is loaded.
- The newsletter sign-up form is visible.
**Test Steps:**
1. Navigate to the newsletter sign-up form.
2. Enter a valid email address in the email field.
3. Submit the form.
4. Verify that the system confirms the successful sign-up and provides instructions on how to unsubscribe from the newsletter.
**Expected Results:**
- The system confirms the successful sign-up and provides clear instructions on how the user can unsubscribe from the newsletter in the future.
**Test Data:**
- Valid email address: "valid_email@example.com"
**Actual Results:**
**Test Status:**

## Test Case ID: TC_005
**Test Description:** Verify that the system securely stores the email addresses of newsletter subscribers.
**Pre-Conditions:**
- The newsletter sign-up process has been completed successfully.
**Test Steps:**
1. Verify that the email address of the newsletter subscriber is stored securely in the system's database or email service.
2. Verify that the email address is not stored in plain text and appropriate security measures are in place (e.g., encryption, hashing).
**Expected Results:**
- The system securely stores the email addresses of newsletter

In [15]:
requirement = "The system must allow users to verify their accounts using their phone number"
result = crew.kickoff(inputs={"requirement": requirement})

 [DEBUG]: == Working Agent: Business Analyst
 [INFO]: == Starting Task: 1. Understand the functional business requirement and relevant context on the requirement The system must allow users to verify their accounts using their phone number.
2. Identify the users or business stakeholders involved in the requirement considering their interests and outcomes relevant to the requirement.
3. Develop a set of agile user stories that wil help to implement the requirement as a software product.
4. Include assumptions, acceptance criteria, error conditions as part of the generated user stories.


> Entering new CrewAgentExecutor chain...
Thought: I now have a good understanding of the requirement and the task at hand. I will carefully craft a comprehensive set of agile user stories to address the "The system must allow users to verify their accounts using their phone number" requirement.

Final Answer:

User Story 1:
As a registered user, I want to be able to verify my account using my phone num

In [16]:
Markdown(result)

## System Test Cases for the Requirement: The system must allow users to verify their accounts using their phone number

### Test Case ID: TC-001
**Test Description:** Verify successful account verification using registered phone number
**Pre-Conditions:**
- The user has a registered account with the system
- The user has a registered phone number associated with their account

**Test Steps:**
1. User navigates to the account verification page
2. User enters their registered phone number
3. System sends a one-time verification code to the user's registered phone number
4. User receives the verification code and enters it into the system
5. System verifies the entered code and grants the user access to their account

**Expected Results:**
- The user can successfully initiate the account verification process
- The system sends the verification code to the user's registered phone number within a reasonable time frame
- The user can successfully enter the received verification code to complete the account verification
- The system verifies the entered code and grants the user access to their account

**Test Data:**
- Registered phone number: +1 555-1234567
- Verification code: 123456

**Actual Results:**
Test Status:

### Test Case ID: TC-002
**Test Description:** Verify error handling for invalid phone number
**Pre-Conditions:**
- The user does not have a registered phone number associated with their account

**Test Steps:**
1. User navigates to the account verification page
2. User enters an invalid phone number
3. System displays an error message and prompts the user to re-enter a valid phone number

**Expected Results:**
- The system detects an invalid phone number and displays an appropriate error message
- The system prompts the user to re-enter a valid phone number

**Test Data:**
- Invalid phone number: 123456789

**Actual Results:**
Test Status:

### Test Case ID: TC-003
**Test Description:** Verify error handling for failed verification code delivery
**Pre-Conditions:**
- The user has a registered phone number associated with their account
- The system is unable to deliver the verification code to the user's phone number

**Test Steps:**
1. User navigates to the account verification page
2. User enters their registered phone number
3. System fails to deliver the verification code to the user's phone number
4. System displays an error message and provides the user with options to retry or contact customer support

**Expected Results:**
- The system is unable to deliver the verification code to the user's phone number
- The system displays an appropriate error message
- The system provides the user with options to retry the verification process or contact customer support

**Test Data:**
- Registered phone number: +1 555-1234567

**Actual Results:**
Test Status:

### Test Case ID: TC-004
**Test Description:** Verify successful phone number update
**Pre-Conditions:**
- The user has a registered account with the system
- The user has a registered phone number associated with their account

**Test Steps:**
1. User navigates to the account settings page
2. User enters a new phone number
3. System sends a verification code to the new phone number
4. User receives the verification code and enters it into the system
5. System verifies the entered code and updates the user's registered phone number

**Expected Results:**
- The user can successfully update their registered phone number in the system
- The system sends a verification code to the new phone number
- The user can enter the received verification code to complete the phone number update
- The system updates the user's account with the new verified phone number

**Test Data:**
- Current registered phone number: +1 555-1234567
- New phone number: +1 555-7654321
- Verification code: 987654

**Actual Results:**
Test Status:

### Test Case ID: TC-005
**Test Description:** Verify error handling for invalid new phone number
**Pre-Conditions:**
- The user has a registered account with the system

**Test Steps:**
1. User navigates to the account settings page
2. User enters an invalid new phone number
3. System displays an error message and prompts the user to re-enter a valid phone number

**Expected Results:**
- The system detects an